In [1]:
import os
import sys
import json
import codecs

import time
from datetime import datetime

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark import SparkFiles
from pyspark.sql.functions import col, lit, length, row_number, when

In [3]:
from lib import spark_utils

In [4]:
spark = spark_utils.get_spark('test222')

25/03/18 18:37:31 WARN Utils: Your hostname, Mac-MD2XX1D4WV.local resolves to a loopback address: 127.0.0.1; using 192.168.11.215 instead (on interface en0)
25/03/18 18:37:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 18:37:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 18:37:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57882)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/

In [5]:
spark

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.width', 4000)

In [7]:
import os
import glob
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, length, row_number, when
from pyspark.sql.functions import avg, count, sum
from pyspark.sql.functions import upper, lower
from pyspark.sql.functions import substring, split
from pyspark.sql import functions
from pyspark.sql.types import IntegerType, StringType, LongType
from pyspark.sql.types import StructType, StructField

In [8]:
def cast_to_int_with_default(value):
    try:
        if value is None:
            return 0  # or any other default value you prefer
        else:
            return int(value)
    except ValueError:
        return 0

from pyspark.sql.functions import udf

udf_cast_to_int_with_default = udf(cast_to_int_with_default, IntegerType())

In [9]:
def get_joined_data_date(spark, input_path, _year, _qurt):
    coverpage_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(
        input_path % (_year, _qurt, 'COVERPAGE.tsv'))

    coverpage_ = coverpage_.withColumns({'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    infotable_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(
        input_path % (_year, _qurt, 'INFOTABLE.tsv'))

    # cusup 合法性检查
    infotable_ = infotable_.withColumn('CUSIP', lower(col('CUSIP')))
    infotable_ = infotable_.filter((infotable_['SSHPRNAMTTYPE'] == 'SH') &
                                   (infotable_['PUTCALL'].isNull()) &
                                   (infotable_['CUSIP'] != '000000000') &
                                   (infotable_['CUSIP'] != '0000000na'))
    infotable_ = infotable_.filter(length(infotable_['CUSIP']) == 9)

    # 仅选取普通股
    infotable_ = infotable_.filter(upper(col("TITLEOFCLASS")).contains("COM"))
    infotable_ = infotable_.withColumns({
        'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    summarypage_ = spark.read.option('delimiter', '\t').option(
        'header', True).csv(input_path % (_year, _qurt, 'SUMMARYPAGE.tsv'))

    summarypage_ = summarypage_.withColumns({
        'YEAR': lit(_year),
        'QUARTER': lit(_qurt)})

    submission_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(input_path % (_year, _qurt, 'SUBMISSION.tsv'))
    submission_ = submission_.withColumns({'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    joined_ = infotable_.join(
        coverpage_, (infotable_['ACCESSION_NUMBER'] == coverpage_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == coverpage_['YEAR']) & (infotable_['QUARTER'] == coverpage_['QUARTER']),
        'left'
    ).join(
        summarypage_, (infotable_['ACCESSION_NUMBER'] == summarypage_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == summarypage_['YEAR']) & (infotable_['QUARTER'] == summarypage_['QUARTER']),
        'left'
    ).join(
        submission_, (infotable_['ACCESSION_NUMBER'] == submission_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == submission_['YEAR']) & (infotable_['QUARTER'] == submission_['QUARTER']),
        'left'
    ).drop(
        coverpage_['YEAR'], coverpage_['QUARTER'], coverpage_['ACCESSION_NUMBER'],
        summarypage_['YEAR'], summarypage_['QUARTER'], summarypage_['ACCESSION_NUMBER'],
        submission_['YEAR'], submission_['QUARTER'], submission_['ACCESSION_NUMBER'])

    joined_ = joined_.withColumns({
        'NAMEOFISSUER': upper(col('NAMEOFISSUER')),
        'FILINGMANAGER_NAME': upper(col('FILINGMANAGER_NAME'))
    })

    # 需要有一个去重判断,投资机构可能多次上传,13HR,13HR-A,...
    window_spec = Window.partitionBy(
        'CUSIP', 'FILINGMANAGER_NAME'
    ).orderBy(col('FILING_DATE').desc(), col('VALUE').desc())

    joined_ = joined_.withColumn('row_number', row_number().over(window_spec))
    joined_ = joined_.filter(col('row_number') == 1).drop('row_number')

    return joined_

In [10]:
def transfer_standard_unit(filter_data, year):
    filter_data = filter_data.withColumns({
        'VALUE': udf_cast_to_int_with_default(col('VALUE')),
        'SSHPRNAMT': udf_cast_to_int_with_default(col('SSHPRNAMT')),
    })
    # 2022年及以前的VALUE单位为千$，2023年及之后为$(实际从1月3号开始)
    filter_data = filter_data.withColumn(
        'VALUE', when(lit(year) >= 2023, col('VALUE')).otherwise(col('VALUE')*1000))
    return filter_data

In [11]:
def filter_data_by_share_value(filter_data, year):
    # 对空值或负值填0
    filter_data = filter_data.withColumns({
        "VALUE": when(col("VALUE").isNull() | (col("VALUE") < 0), 0).otherwise(col("VALUE")),
        "SSHPRNAMT": when(col("SSHPRNAMT").isNull() | (col("SSHPRNAMT") < 0), 0).otherwise(col("SSHPRNAMT")),
    })
    # 以股票为Key统计总被交易价值、效果量
    ticker_value = filter_data.groupby(['CUSIP']).agg(
        sum(col('VALUE')).alias('VALUE'),
        sum(col('SSHPRNAMT')).alias('SSHPRNAMT')
    ).withColumn(
        'VPSSH', col('VALUE') / col('SSHPRNAMT')
    ).filter(
        (col('VPSSH') > 0.00000000000001) & (col('VPSSH') < 100000000000.)
    )

    ticker_value = ticker_value.withColumnRenamed(
        'CUSIP', 'CUSIP_ticker'
    ).withColumnRenamed(
        'VALUE', 'VALUE_ticker'
    ).withColumnRenamed(
        'SSHPRNAMT', 'SSHPRNAMT_ticker'
    )

    # 如果基金持有成本明显偏离总平均成本，认为是脏数据，过滤掉
    filter_data = filter_data.join(
        ticker_value, filter_data['CUSIP'] == ticker_value['CUSIP_ticker'], 'left'
    ).drop('CUSIP_ticker', 'VALUE_ticker', 'SSHPRNAMT_ticker')

    filter_data = filter_data.filter(
        (col('VALUE') / col('SSHPRNAMT') > 0.1 * col('VPSSH'))
        & (col('VALUE') / col('SSHPRNAMT') < 10 * col('VPSSH')))

    filter_data = filter_data.drop('VPSSH')

    return filter_data



In [12]:
input_path = '/Users/liuda/Library/CloudStorage/Dropbox/shareit/code/trading/data/hedge/%sq%s_form13f/%s'
cur_year='2024'
cur_qurt='3'
pre_year='2024'
pre_qurt='2'

In [13]:
joined_cur = get_joined_data_date(spark, input_path, cur_year, cur_qurt)
joined_cur = transfer_standard_unit(joined_cur, cur_year)
joined_cur = filter_data_by_share_value(joined_cur, cur_year)

In [14]:
joined_cur.groupby(
    ['CUSIP', 'FILINGMANAGER_NAME']
).agg(
    {'VALUE': 'sum'}
).limit(10).toPandas()
# 已经确认，没有CUSIP, FILINGMANAGER_NAME count(ACCESSION_NUMBER)全部为1了，不存在重复数据了 

,CUSIP,FILINGMANAGER_NAME,sum(VALUE)
0,005098108,BROWN BROTHERS HARRIMAN & CO,60941
1,005098108,COMERICA BANK,355510
2,005098108,CREATIVE PLANNING,456698
3,005098108,DEUTSCHE BANK AG\,840475
4,005098108,ENGINEERS GATE MANAGER LP,1453502
5,005098108,ENVESTNET ASSET MANAGEMENT INC,8307484
6,005098108,FIRST TRUST ADVISORS LP,4480101
7,005098108,"HIGHTOWER ADVISORS, LLC",282000
8,005098108,INTEGRATED WEALTH CONCEPTS LLC,282613
9,005098108,KEY FINANCIAL INC,6792


In [15]:
# TSM: 874039100
df = joined_cur.filter((col('CUSIP') == '874039100') & (col('FILINGMANAGER_NAME')== 'ACADEMY CAPITAL MANAGEMENT INC/TX'))

df.select('VALUE', 'FILINGMANAGER_NAME', 'CUSIP').limit(10).toPandas()

,VALUE,FILINGMANAGER_NAME,CUSIP
0,10649165,ACADEMY CAPITAL MANAGEMENT INC/TX,874039100


In [21]:
df = joined_cur.filter((col('CUSIP') == '874039100'))
df.agg(sum('VALUE')).toPandas()

,sum(VALUE)
0,2387849256


In [20]:
# 2387849256

In [18]:
joined_pre = get_joined_data_date(spark, input_path, pre_year, pre_qurt)   # 过滤CUSIP

joined_pre = joined_pre.select(['NAMEOFISSUER', 'FILINGMANAGER_NAME', 'CUSIP',
                                'YEAR', 'QUARTER', 'VALUE',
                                'SSHPRNAMTTYPE', 'SSHPRNAMT',
                                'FILING_DATE'])

# 统一VALUE单位
joined_pre = transfer_standard_unit(joined_pre, pre_year)

# 统计股票的均价，如果基金成本明显异于均价，认为是异常值,过滤
joined_pre = filter_data_by_share_value(joined_pre, pre_year)

joined_pre = joined_pre.withColumnRenamed(
    "VALUE", "PREVALUE").withColumnRenamed(
    'YEAR', 'PREYEAR').withColumnRenamed(
    'QUARTER', 'PREQUARTER').withColumnRenamed(
    'VALUE', 'PREVALUE').withColumnRenamed(
    'SSHPRNAMTTYPE', 'PRESSHPRNAMTTYPE').withColumnRenamed(
    'SSHPRNAMT', 'PRESSHPRNAMT').withColumnRenamed(
    'FILING_DATE', 'PRE_FILING_DATE')

In [18]:
joined_data = joined_cur.join(
    joined_pre, (joined_cur['FILINGMANAGER_NAME'] == joined_pre['FILINGMANAGER_NAME']) &
    (joined_cur['CUSIP'] == joined_pre['CUSIP']),
    'left'
).drop(
    joined_pre['FILINGMANAGER_NAME'], joined_pre['NAMEOFISSUER'], joined_pre['CUSIP']
).withColumn('ADDITIONALINFORMATION', substring('ADDITIONALINFORMATION', 0, 2000))

joined_data = joined_data.withColumns({
    "VALUE": when(col("VALUE").isNull() | (col("VALUE") < 0), 0).otherwise(col("VALUE")),
    "PREVALUE": when(col("PREVALUE").isNull() | (col("PREVALUE") < 0), 0).otherwise(col("PREVALUE")),
    "SSHPRNAMT": when(col("SSHPRNAMT").isNull() | (col("SSHPRNAMT") < 0), 0).otherwise(col("SSHPRNAMT")),
    "PRESSHPRNAMT": when(col("PRESSHPRNAMT").isNull() | (col("PRESSHPRNAMT") < 0), 0).otherwise(col("PRESSHPRNAMT"))
})

In [21]:
joined_data.persist()

DataFrame[ACCESSION_NUMBER: string, INFOTABLE_SK: string, NAMEOFISSUER: string, TITLEOFCLASS: string, CUSIP: string, FIGI: string, VALUE: int, SSHPRNAMT: int, SSHPRNAMTTYPE: string, PUTCALL: string, INVESTMENTDISCRETION: string, OTHERMANAGER: string, VOTING_AUTH_SOLE: string, VOTING_AUTH_SHARED: string, VOTING_AUTH_NONE: string, YEAR: string, QUARTER: string, REPORTCALENDARORQUARTER: string, ISAMENDMENT: string, AMENDMENTNO: string, AMENDMENTTYPE: string, CONFDENIEDEXPIRED: string, DATEDENIEDEXPIRED: string, DATEREPORTED: string, REASONFORNONCONFIDENTIALITY: string, FILINGMANAGER_NAME: string, FILINGMANAGER_STREET1: string, FILINGMANAGER_STREET2: string, FILINGMANAGER_CITY: string, FILINGMANAGER_STATEORCOUNTRY: string, FILINGMANAGER_ZIPCODE: string, REPORTTYPE: string, FORM13FFILENUMBER: string, CRDNUMBER: string, SECFILENUMBER: string, PROVIDEINFOFORINSTRUCTION5: string, ADDITIONALINFORMATION: string, OTHERINCLUDEDMANAGERSCOUNT: string, TABLEENTRYTOTAL: string, TABLEVALUETOTAL: string

In [20]:
joined_data.filter(col('CUSIP') == '007903107').limit(3).toPandas()

,ACCESSION_NUMBER,INFOTABLE_SK,NAMEOFISSUER,TITLEOFCLASS,CUSIP,FIGI,VALUE,SSHPRNAMT,SSHPRNAMTTYPE,PUTCALL,INVESTMENTDISCRETION,OTHERMANAGER,VOTING_AUTH_SOLE,VOTING_AUTH_SHARED,VOTING_AUTH_NONE,YEAR,QUARTER,REPORTCALENDARORQUARTER,ISAMENDMENT,AMENDMENTNO,AMENDMENTTYPE,CONFDENIEDEXPIRED,DATEDENIEDEXPIRED,DATEREPORTED,REASONFORNONCONFIDENTIALITY,FILINGMANAGER_NAME,FILINGMANAGER_STREET1,FILINGMANAGER_STREET2,FILINGMANAGER_CITY,FILINGMANAGER_STATEORCOUNTRY,FILINGMANAGER_ZIPCODE,REPORTTYPE,FORM13FFILENUMBER,CRDNUMBER,SECFILENUMBER,PROVIDEINFOFORINSTRUCTION5,ADDITIONALINFORMATION,OTHERINCLUDEDMANAGERSCOUNT,TABLEENTRYTOTAL,TABLEVALUETOTAL,ISCONFIDENTIALOMITTED,FILING_DATE,SUBMISSIONTYPE,CIK,PERIODOFREPORT,PREYEAR,PREQUARTER,PREVALUE,PRESSHPRNAMTTYPE,PRESSHPRNAMT,PRE_FILING_DATE
0,0001415889-24-019913,102529364,ADVANCED MICRO DEVICES INC,COM,007903107,None,15897,98,SH,None,SOLE,None,0,0,98,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,0,None,0,None
1,0001415889-24-020016,102574540,ADVANCED MICRO DEVICES INC,COMM,007903107,BBG001S5NN36,13268778,81800,SH,None,SOLE,None,81800,0,0,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"ONCE CAPITAL MANAGEMENT, LLC",450 PARK AVENUE,27TH FLOOR,NEW YORK,NY,10022,13F HOLDINGS REPORT,028-22160,None,801-120989,N,None,0,24,267160958,N,24-JUL-2024,13F-HR,0001712892,30-JUN-2024,None,None,0,None,0,None
2,0001415889-24-021128,103861698,ADVANCED MICRO DEVICES INC,Common,007903107,None,408847035,2520517,SH,None,DFND,None,2212402,0,308114,2024,3,30-JUN-2024,None,None,None,None,None,None,None,NEUBERGER BERMAN GROUP LLC,1290 AVENUE OF THE AMERICAS,None,NEW YORK,NY,10104,13F HOLDINGS REPORT,028-13573,None,None,N,None,9,3283,121843930448,N,13-AUG-2024,13F-HR,0001465109,30-JUN-2024,None,None,0,None,0,None


In [19]:
joined_data = joined_cur.join(
    joined_pre, (joined_cur['FILINGMANAGER_NAME'] == joined_pre['FILINGMANAGER_NAME']) &
    (joined_cur['CUSIP'] == joined_pre['CUSIP']),
    'outer'  # 这里因为要求PREVALUE，所以应该为outer
).drop(
    joined_pre['FILINGMANAGER_NAME'], joined_pre['NAMEOFISSUER'], joined_pre['CUSIP']
).withColumn('ADDITIONALINFORMATION', substring('ADDITIONALINFORMATION', 0, 2000))


In [20]:
joined_data.limit(10).toPandas()

25/03/18 18:41:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,ACCESSION_NUMBER,INFOTABLE_SK,NAMEOFISSUER,TITLEOFCLASS,CUSIP,FIGI,VALUE,SSHPRNAMT,SSHPRNAMTTYPE,PUTCALL,INVESTMENTDISCRETION,OTHERMANAGER,VOTING_AUTH_SOLE,VOTING_AUTH_SHARED,VOTING_AUTH_NONE,YEAR,QUARTER,REPORTCALENDARORQUARTER,ISAMENDMENT,AMENDMENTNO,AMENDMENTTYPE,CONFDENIEDEXPIRED,DATEDENIEDEXPIRED,DATEREPORTED,REASONFORNONCONFIDENTIALITY,FILINGMANAGER_NAME,FILINGMANAGER_STREET1,FILINGMANAGER_STREET2,FILINGMANAGER_CITY,FILINGMANAGER_STATEORCOUNTRY,FILINGMANAGER_ZIPCODE,REPORTTYPE,FORM13FFILENUMBER,CRDNUMBER,SECFILENUMBER,PROVIDEINFOFORINSTRUCTION5,ADDITIONALINFORMATION,OTHERINCLUDEDMANAGERSCOUNT,TABLEENTRYTOTAL,TABLEVALUETOTAL,ISCONFIDENTIALOMITTED,FILING_DATE,SUBMISSIONTYPE,CIK,PERIODOFREPORT,PREYEAR,PREQUARTER,PREVALUE,PRESSHPRNAMTTYPE,PRESSHPRNAMT,PRE_FILING_DATE
0,0001415889-24-019913,102529384,A-MARK PRECIOUS METALS INC,COM,00181t107,None,1492322,46102,SH,None,SOLE,None,0,0,46102,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
1,0001415889-24-019913,102529373,ALCOA CORP,COM,013872106,None,3394,85,SH,None,SOLE,None,0,0,85,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
2,0001415889-24-019913,102529375,ALEXANDRIA REAL ESTATE EQ INC,COM,015271109,None,2339,20,SH,None,SOLE,None,0,0,20,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
3,0001415889-24-019913,102529501,CENCORA INC,COM,03073e105,None,7210,32,SH,None,SOLE,None,0,0,32,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
4,0001415889-24-019913,102529411,ARCHER DANIELS MIDLAND CO,COM,039483102,None,24180,400,SH,None,SOLE,None,0,0,400,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
5,0001415889-24-019913,102529425,AUTODESK INC,COM,052769106,None,4207,17,SH,None,SOLE,None,0,0,17,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
6,0001415889-24-019913,102529432,AVIS BUDGET GROUP,COM,053774105,None,314,3,SH,None,SOLE,None,0,0,3,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
7,0001415889-24-019913,102529438,BANK MARIN BANCORP,COM,063425102,None,381436,23560,SH,None,SOLE,None,0,0,23560,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,None,N,None,0,925,202066594,N,23-JUL-2024,13F-HR,0000702007,30-JUN-2024,None,None,NaN,None,NaN,None
8,0001415889-24-019913,102529440,BANK NEW YORK MELLON CORP,COM,064058100,None,475706,7943,SH,None,SOLE,None,0,0,7943,2024,3,30-JUN-2024,None,None,None,None,None,None,None,"CALDWELL SUTTER CAPITAL, INC.",PO BOX 190,None,SAUSALITO,CA,94966-0190,13F HOLDINGS REPORT,028-22901,None,